In [1]:
import pandas as pd
import csv
import os
from operator import itemgetter

In [2]:
code_path = os.getcwd()
project_file_path = code_path.split("chembl")[0]
training_files_path = os.path.join(project_file_path, "training_files")
training_files_path

'/home/hayriye/DEEPScreen2.2/training_files'

In [3]:
chembl_filtered_bioact_fl = os.path.join(training_files_path, "chembl32_raw_filtered_sp_b_pchembl_data.csv4xls")
raw_dataset_df = pd.read_csv(chembl_filtered_bioact_fl, sep=';', quoting=csv.QUOTE_ALL, low_memory=False)
raw_dataset_df.head()

,Target_CHEMBL_ID,Compound_CHEMBL_ID,pchembl_value,Assay_CHEMBL_ID,assay_type,standard_type,standard_value,standard_units,Assay Taxonomy,TD Tax ID,confidence_score,target_type,src_compound_id,src_assay_id,src_id,src_description,standard_relation,activity_comment,description,year
0,CHEMBL3066,CHEMBL2370208,-10.61,CHEMBL712411,B,Ki,40738027780411220000.0000000000000000000000000...,nM,NaN,10116,8,SINGLE PROTEIN,NaN,NaN,1,Scientific Literature,=,NaN,In vitro binding affinity for rat pituitary LH...,1993.0
1,CHEMBL261,CHEMBL7204,-4.89,CHEMBL858760,B,Ki,77624711662869.280000000000000000000000000000,nM,NaN,9606,8,SINGLE PROTEIN,NaN,NaN,1,Scientific Literature,=,NaN,Inhibition of human carbonic anhydrase I (CAI),2003.0
2,CHEMBL261,CHEMBL6705,-4.66,CHEMBL858760,B,Ki,45708818961487.520000000000000000000000000000,nM,NaN,9606,8,SINGLE PROTEIN,NaN,NaN,1,Scientific Literature,=,NaN,Inhibition of human carbonic anhydrase I (CAI),2003.0
3,CHEMBL261,CHEMBL21,-4.45,CHEMBL858760,B,Ki,28183829312644.490000000000000000000000000000,nM,NaN,9606,8,SINGLE PROTEIN,NaN,NaN,1,Scientific Literature,=,NaN,Inhibition of human carbonic anhydrase I (CAI),2003.0
4,CHEMBL261,CHEMBL6852,-4.40,CHEMBL858760,B,Ki,25118864315095.820000000000000000000000000000,nM,NaN,9606,8,SINGLE PROTEIN,NaN,NaN,1,Scientific Literature,=,NaN,Inhibition of human carbonic anhydrase I (CAI),2003.0


In [6]:
def create_preprocessed_bioact_file(training_files_path, chembl_filtered_bioact_fl, chembl_version):
    
    raw_dataset_df = pd.read_csv(chembl_filtered_bioact_fl, sep=";", index_col=False, quoting=csv.QUOTE_ALL, low_memory=False)
    # keys are compound protein pairs values are list of bioactivities
    annot_dict = dict()
    for ind, row in raw_dataset_df.iterrows():
        chembl_tid = row["Target_CHEMBL_ID"]
        chembl_cid = row["Compound_CHEMBL_ID"]
        standard_units = row["standard_units"]
        row["year"] = row["year"]

        if standard_units in ["uM", "nM", "M"]:
            if standard_units == "nM":
                row["standard_value"] = round(float(row["standard_value"]) / pow(10,3), 3)
            elif standard_units == "M":
                row["standard_value"] = round(float(row["standard_value"]) / pow(10,6), 3)
            else:
                row["standard_value"] = round(float(row["standard_value"]), 3)
            # standard_units = "uM"
            row["standard_units"] = "uM"
            try:
                annot_dict["{},{}".format(chembl_tid, chembl_cid)].append(list(row))
            except:
                annot_dict["{},{}".format(chembl_tid, chembl_cid)] = [list(row)]
    
    out_fl = open("{}/{}_preprocessed_filtered_bioactivity_dataset.tsv".format(training_files_path, chembl_version), "w")
    out_fl.write("\t".join(list(raw_dataset_df.columns)))
    
    for key in annot_dict.keys():
        if len(annot_dict[key])>1:
            median_std_val = 0.0

            annot_dict[key]  = sorted(annot_dict[key], key=itemgetter(6))
            # print(annot_dict[key])

            if len(annot_dict[key])%2==1:
                median = int(len(annot_dict[key])/2)
                median_bioactivity = annot_dict[key][median]
                out_fl.write("\n" + "\t".join([str(col) for col in median_bioactivity]))

            else:
                median = int(len(annot_dict[key])/2)

                median_std_val = (annot_dict[key][median][6]+annot_dict[key][median-1][6])/2
                annot_dict[key][median][6] = median_std_val
                median_bioactivity = annot_dict[key][median]
                out_fl.write("\n" + "\t".join([str(col) for col in median_bioactivity]))
        else:

            out_fl.write("\n"+"\t".join([str(col) for col in annot_dict[key][0]]))

    out_fl.close()

In [7]:
create_preprocessed_bioact_file(training_files_path, chembl_filtered_bioact_fl, "chembl32")